In [1]:
import json
import os
import os.path as osp
from glob import glob
import re
import copy
import warnings
import argparse
import cv2

import numpy as np
import pandas as pd
import math

from tqdm import tqdm

import time

from torchvision.datasets.video_utils import VideoClips

# warnings.filterwarnings(action='ignore')

### Load JSON File

In [2]:
date = "20211217" #original videos sorted by dae
# using the imputated annotation file`
json_file_path = "D:/Dataset_BENG/main_labels/tol45" #new bounding box coordinates
video_path= "D:/Dataset_BENG/main_videos"

In [3]:
anno_file_list=glob(osp.join(json_file_path, date, '*'))
anno_file_list

['D:/Dataset_BENG/main_labels/tol45\\20211217\\pig_hampyeong_cam3_20211217_073449.json',
 'D:/Dataset_BENG/main_labels/tol45\\20211217\\pig_hampyeong_cam3_20211217_125957.json',
 'D:/Dataset_BENG/main_labels/tol45\\20211217\\pig_hampyeong_cam4_20211217_073552.json',
 'D:/Dataset_BENG/main_labels/tol45\\20211217\\pig_hampyeong_cam4_20211217_130059.json']

In [4]:
'''
NEED TO CHANGE THE INDEX here manually: anno_file_list[i]
'''
json_path = anno_file_list[3]
json_path

'D:/Dataset_BENG/main_labels/tol45\\20211217\\pig_hampyeong_cam4_20211217_130059.json'

In [5]:
with open(json_path, 'r') as f:
    json_file=json.load(f)

#### Get the original bounding box coordinates of each frame for each pig

In [6]:
json_copy = copy.deepcopy(json_file)

In [7]:
all_pig_ids = []
for frame in json_copy['label_info']['annotations']:
    for pig in frame:
        id_ = pig['object_id']
        if id_ not in all_pig_ids: #prevent appending same ID but in different frames
            all_pig_ids.append(id_)

In [9]:
# for all pigs
group_N_frames = {}
for id_ in all_pig_ids:
    group_N_frames[id_] = []
for frame in json_copy['label_info']['annotations']:
    for pig in frame:
        id_ = pig['object_id']
        bbox_c = pig['bbox']
#         width = bbox_c[2] - bbox_c[0]
#         height = bbox_c[3] - bbox_c[1]
#         if id_ in all_pig_ids:
#             if sum(bbox_c) > 0:
        group_N_frames[id_].append(bbox_c)

#### Get the MAXIMUM width and height of the bounding box for each pig

In [12]:
max_width_height = {}
for id_, bbox in group_N_frames.items():
    all_width = []
    all_height = []
#     max_coords = np.max(bbox, axis=0)
    for coord in bbox:
        width = coord[2] - coord[0]
        height = coord[3] - coord[1]
        all_width.append(width)
        all_height.append(height)
    max_width = np.max(all_width)
    max_height = np.max(all_height)
    max_width_height[id_] = (max_width, max_height)
#     max_width_height[id_] = (max_width, max_height, max_coords)

#### Expand the bounding box

In [13]:
for frame in json_copy['label_info']['annotations']:
    
    for pig in frame:
        
        pig_id = pig['object_id']
        
        if sum(pig['bbox']) > 0:
            x1, y1, x2, y2 = pig['bbox']            
            
            centre_x = int((x1 + x2)/2)
            centre_y = int((y1 + y2)/2)

            w, h = max_width_height[pig_id][0], max_width_height[pig_id][1]
            w, h = int(w*1.2), int(h*1.2)
            
            x1 = max(0, centre_x - int(w/2))
            if x1 == 0:
                x2 = x1 + w
            else: x2 = min(1920, centre_x + int(w/2))
                
            if x2 == 1920:
                x1 = 1920 - w
                
            y1 = max(0, centre_y - int(h/2))
            if y1 == 0:
                y2 = y1 + h
            else: y2 = min(1080, centre_y + int(h/2))
            
            if y2 == 1080:
                y1 = 1080 - h
                   
            pig['bbox'] = [x1, y1, x2, y2]  
            
#             x_dist = int((x2-x1)/4)
#             y_dist = int((y2-y1)/4)
#             x1 = max(0, x1 - x_dist)
#             x2 = min(1920, x2 + x_dist) #max width of video
#             y1 = max(0, y1 - y_dist)
#             y2 = min(1080, y2 + y_dist) #max height of video

#### New list of BBOX for each pig

In [14]:
frames_by_pig = {}
for id_ in all_pig_ids:
    frames_by_pig[id_] = []
for frame in json_copy['label_info']['annotations']:
    for pig in frame:
        id_ = pig['object_id']
        bbox_c = pig['bbox']
        frames_by_pig[id_].append(bbox_c)

In [ ]:
# bbox_info = pd.DataFrame(columns=['id', 'width_frame', 'height_frame', 'max_width', 'max_height'])
# for id_, bbox in tqdm(group_15frames.items()):
#     max_width = 0
#     max_height = 0
#     for frame_num, coord in enumerate(bbox):
#     for i, j in enumerate(playful_all_frames[99]):
#         width = coord[2] - coord[0]
#         height = coord[3] - coord[1]
#         if width > max_width:
#             max_width = width
#             max_width_idx = frame_num
#         if height > max_height:
#             max_height = height
#             max_height_idx = frame_num
#     bbox_info = bbox_info.append(pd.Series([id_, max_width_idx, max_height_idx, max_width, max_height], index=bbox_info.columns),ignore_index=True)
# bbox_info = bbox_info.astype(int)
# print("Max width is %d at frame %d"%(max_width, max_width_idx))
# print("Max height is %d at frame %d"%(max_height, max_height_idx))

#### Extract short clips one by one based on pig ID

In [15]:
# video_file = 'pig_hampyeong_cam3_20211216_180424.avi'
video_file = os.path.split(json_path)[1].replace('json', 'avi')
video_file_path = os.path.join(video_path, date, video_file)
# capture = cv2.VideoCapture(video_file_path)
time = video_file.split('_')[-1][:-4]

In [16]:
video_file_path

'D:/Dataset_BENG/main_videos\\20211217\\pig_hampyeong_cam4_20211217_130059.avi'

In [ ]:
# if not capture.isOpened():
#     print("\n Video file does not exist. (%s)"% video_file)

# print("Video Open: %s"% video_file)


# w_frame, h_frame = int(capture.get(cv2.CAP_PROP_FRAME_WIDTH)), int(capture.get(cv2.CAP_PROP_FRAME_HEIGHT))
# frames = int(capture.get(cv2.CAP_PROP_FRAME_COUNT))

# video_fps= int(capture.get(cv2.CAP_PROP_FPS))
# print("Number of frames per second:", video_fps)
# print("Total number of frames:", frames)

In [25]:
# loop pig by pig
for pig_id in all_pig_ids:

    capture = cv2.VideoCapture(video_file_path)
    
    if not capture.isOpened():
        print("\n Video file does not exist. (%s)"% video_file)
    print("Video Open: %s"% video_file)
    
    
    w_frame, h_frame = int(capture.get(cv2.CAP_PROP_FRAME_WIDTH)), int(capture.get(cv2.CAP_PROP_FRAME_HEIGHT))
    frames = int(capture.get(cv2.CAP_PROP_FRAME_COUNT))
    video_fps= int(capture.get(cv2.CAP_PROP_FPS))
    
    # Pre-define the cropping size
    w, h = max_width_height[pig_id][0], max_width_height[pig_id][1]
    w, h = int(w*1.2), int(h*1.2)
    
    # Initialize frame counter
    cnt = 0

    # output
    fourcc = cv2.VideoWriter_fourcc(*'DIVX')
    datetime_path = "D:/Dataset_BENG/short_clips/{}".format(date+time)
    if not osp.exists(datetime_path):
        os.makedirs(datetime_path)
    out = cv2.VideoWriter("D:/Dataset_BENG/short_clips/{}/{}.avi".format(date+time, pig_id), fourcc, video_fps,(w, h))


    # Now we start
    while(capture.isOpened()):
        ret, frame = capture.read()

        try:
            bbox = frames_by_pig[pig_id][cnt]
        except:
            break


        cnt += 1 # Counting frames

        # Avoid problems when video finish
        if ret:
            # Croping the frame
            crop_frame = frame[bbox[1]:bbox[3], bbox[0]:bbox[2]]
            
            # Percentage
            xx = cnt*100/frames
            print(int(xx),'%')

            # Saving from the desired frames
            #if 15 <= cnt <= 90:
            #    out.write(crop_frame)

            # I see the answer now. Here you save all the video
            out.write(crop_frame)
                

            # Just to see the video in real time          
    #         cv2.imshow('frame',frame)
    #         cv2.imshow('croped',crop_frame)

#             if cv2.waitKey(1) & 0xFF == ord('q'):
#                 break
       
        else:
            break


capture.release()
out.release()
cv2.destroyAllWindows()

Video Open: pig_hampyeong_cam4_20211217_130059.avi
0 %
0 %
0 %
0 %
0 %
0 %
0 %
0 %
0 %
0 %
0 %
0 %
0 %
0 %
0 %
0 %
0 %
1 %
1 %
1 %
1 %
1 %
1 %
1 %
1 %
1 %
1 %
1 %
1 %
1 %
1 %
1 %
1 %
1 %
1 %
2 %
2 %
2 %
2 %
2 %
2 %
2 %
2 %
2 %
2 %
2 %
2 %
2 %
2 %
2 %
2 %
2 %
2 %
3 %
3 %
3 %
3 %
3 %
3 %
3 %
3 %
3 %
3 %
3 %
3 %
3 %
3 %
3 %
3 %
3 %
3 %
4 %
4 %
4 %
4 %
4 %
4 %
4 %
4 %
4 %
4 %
4 %
4 %
4 %
4 %
4 %
4 %
4 %
4 %
5 %
5 %
5 %
5 %
5 %
5 %
5 %
5 %
5 %
5 %
5 %
5 %
5 %
5 %
5 %
5 %
5 %
5 %
6 %
6 %
6 %
6 %
6 %
6 %
6 %
6 %
6 %
6 %
6 %
6 %
6 %
6 %
6 %
6 %
6 %
6 %
7 %
7 %
7 %
7 %
7 %
7 %
7 %
7 %
7 %
7 %
7 %
7 %
7 %
7 %
7 %
7 %
7 %
7 %
8 %
8 %
8 %
8 %
8 %
8 %
8 %
8 %
8 %
8 %
8 %
8 %
8 %
8 %
8 %
8 %
8 %
8 %
9 %
9 %
9 %
9 %
9 %
9 %
9 %
9 %
9 %
9 %
9 %
9 %
9 %
9 %
9 %
9 %
9 %
9 %
10 %
10 %
10 %
10 %
10 %
10 %
10 %
10 %
10 %
10 %
10 %
10 %
10 %
10 %
10 %
10 %
10 %
10 %
11 %
11 %
11 %
11 %
11 %
11 %
11 %
11 %
11 %
11 %
11 %
11 %
11 %
11 %
11 %
11 %
11 %
11 %
12 %
12 %
12 %
12 %
12 %
12 %
12 %
12 %
12 %
12 %
12 

92 %
93 %
93 %
93 %
93 %
93 %
93 %
93 %
93 %
93 %
93 %
93 %
93 %
93 %
93 %
93 %
93 %
93 %
93 %
94 %
94 %
94 %
94 %
94 %
94 %
94 %
94 %
94 %
94 %
94 %
94 %
94 %
94 %
94 %
94 %
94 %
94 %
95 %
95 %
95 %
95 %
95 %
95 %
95 %
95 %
95 %
95 %
95 %
95 %
95 %
95 %
95 %
95 %
95 %
95 %
96 %
96 %
96 %
96 %
96 %
96 %
96 %
96 %
96 %
96 %
96 %
96 %
96 %
96 %
96 %
96 %
96 %
96 %
97 %
97 %
97 %
97 %
97 %
97 %
97 %
97 %
97 %
97 %
97 %
97 %
97 %
97 %
97 %
97 %
97 %
97 %
98 %
98 %
98 %
98 %
98 %
98 %
98 %
98 %
98 %
98 %
98 %
98 %
98 %
98 %
98 %
98 %
98 %
98 %
99 %
99 %
99 %
99 %
99 %
99 %
99 %
99 %
99 %
99 %
99 %
99 %
99 %
99 %
99 %
99 %
99 %
99 %
100 %


<br>

<br>

<br>

### Not important

In [10]:
'''
Get the maximum width and height of the bounding box among all pigs
'''
# all_width = []
# all_height = []
# for bbox in group_N_frames.values():
#     for coord in bbox:
#         width = coord[2] - coord[0]
#         height = coord[3] - coord[1]
#         all_width.append(width)
#         all_height.append(height)
        
        
# cliptrim_width = np.max(all_width)
# cliptrim_height = np.max(all_height)
# print("Max width is %d"%(cliptrim_width))
# print("Max height is %d"%(cliptrim_height))

#### Find the index of 60-frames-long clips with ALL (0,0,0,0) coordinates

In [ ]:
# num_frames = 60
# trim_by_frames = {}
# for id_ in frames_by_pig:
#     trim_by_frames[id_]= []
#     for idx in range(1800):
#         if (idx+1)%num_frames == 0:
#             trim_by_frames[id_].append(frames_by_pig[id_][idx-num_frames+1:idx+1])

In [ ]:
# no_pixel_frame = pd.DataFrame(columns=['pig_id', 'start_frames'])
# for id_ in  all_pig_ids:
#     empty_frame = []
#     for idx, bbox in enumerate(trim_by_frames[id_]):
#         if np.sum(bbox) == 0:
#             empty_frame.append((idx+1)*num_frames)
#     temp = {'pig_id': id_, 'start_frames': empty_frame}
#     no_pixel_frame = pd.concat([no_pixel_frame, pd.DataFrame(temp)], ignore_index=True)